In [1]:
import modelx as mx
import numpy as np

model = mx.new_model()                  # Create a new Model named "Model1"
space = model.new_space("MonteCarlo")   # Create a UserSpace named "MonteCralo"

# Define names in MonteCarlo
space.np = np
space.M = 10000     # Number of scenarios
space.T = 3         # Time to maturity in years
space.N = 36        # Number of time steps
space.S0 = 100      # S(0): Stock price at t=0
space.r = 0.05      # Risk Free Rate
space.sigma = 0.2   # Volatility
space.K = 110       # Option Strike


# Define Cells objects from function definitions
@mx.defcells
def std_norm_rand():
    gen = np.random.default_rng(1234)
    return gen.standard_normal(size=(N, M))


@mx.defcells
def S(i):
    """Stock price at time t_i"""
    dt = T/N; t = dt * i
    if i == 0:
        return np.full(shape=M, fill_value=S0)
    else:
        epsilon = std_norm_rand()[i-1]
        return S(i-1) * np.exp((r - 0.5 * sigma**2) * dt + sigma * epsilon * dt**0.5)


@mx.defcells
def CallOption():
    """Call option price by Monte-Carlo"""
    return np.average(np.maximum(S(N) - K, 0)) * np.exp(-r*T)

In [2]:
@mx.defcells
def BlackScholesCall():
    """Call option price by Black-Scholes-Merton"""
    from scipy.stats import norm
    e = np.exp
    N = norm.cdf

    d1 = (np.log(S0/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S0 * N(d1) - K * e(-r*T) * N(d2)  

In [3]:
BlackScholesCall()

16.210871364283975

In [4]:
space.K = 100

In [5]:
S(space.N)

array([ 78.58406132,  59.01504804, 115.148291  , ..., 155.39335662,
        74.7907511 , 137.82730703])

In [6]:
CallOption()

20.96156962064

In [7]:
space.parameters = ("r", "sigma")

In [8]:
space[0.03, 0.15].CallOption()

14.812014828333284

In [9]:
space[0.06, 0.4].CallOption()

33.90481014639403